In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import simweights
import pickle
import os, sys
import re
import numpy as np
import matplotlib as mat
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
import pandas as pd
import tables
import h5py
import math
from scipy.stats import mstats
import matplotlib as mpl
import matplotlib.font_manager as font_manager


In [3]:
sys.path.append("/data/user/tvaneede/GlobalFit/reco_processing/notebooks/weighting")
from weights import *
from utils import *
from datasets import datasets
from selections import selection_mask
from fonts import *
from plot_utils import *

In [4]:
livetime_yr = 11.687
livetime_s  = livetime_yr * 365.25 * 24 * 3600 # 11.687 year

In [5]:
dataset_names = ["neha_track_AllHESE", "neha_cascade_AllHESE", "neha_doublecascade_AllHESE"]

files = {}

for dataset_name in dataset_names: files[dataset_name] = datasets[dataset_name]

In [6]:
version = "neha_AllHESE"
main_plotting_path = f"/data/user/tvaneede/GlobalFit/reco_processing/notebooks/debug_spice_ftp_weights/output/{version}"
os.system(f"mkdir -p {main_plotting_path}")

0

In [ ]:
# open the files
for channel,files_channel in files.items():
    print(f"----- Extracting files for channel {channel}")
    for flavor in files_channel:
        files_channel[flavor]['hdf_file'] = pd.HDFStore(files_channel[flavor]['file_path'],'r')
        # files_channel[flavor]['variables'] = get_variables_neha( files_channel[flavor]['hdf_file'] )
        files_channel[flavor]['weighter'] = simweights.NuGenWeighter( files_channel[flavor]['hdf_file'] ,nfiles=files_channel[flavor]['nfiles'])


----- Extracting files for channel neha_track_AllHESE
----- Extracting files for channel neha_cascade_AllHESE
----- Extracting files for channel neha_doublecascade_AllHESE


In [11]:
keys_to_merge = {
    "NuE" : ["NuE_E2_1", "NuE_E3_1","NuE_E2_2", "NuE_E3_2"],
    "NuMu" : ["NuMu_E2_1", "NuMu_E3_1","NuMu_E2_2", "NuMu_E3_2"],
    "NuTau" : ["NuTau_E2_1", "NuTau_E3_1","NuTau_E2_2", "NuTau_E3_2"],
    "NuAll" : ['NuE', "NuMu", "NuTau"],
}

for channel,files_channel in files.items():
    for new_key in keys_to_merge:
        files_channel[new_key] = {}
        files_channel[new_key]['variables'] = {}
        files_channel[new_key]['weighter'] = None

        for key in keys_to_merge[new_key]:

            # merge the weighters
            if files_channel[new_key]['weighter'] == None:
                files_channel[new_key]['weighter'] = files_channel[key]['weighter']
            else: files_channel[new_key]['weighter'] += files_channel[key]['weighter']

        #     # merge the variables
        #     for variable in files_channel[key]['variables']:
        #         if variable not in files_channel[new_key]['variables']:
        #             files_channel[new_key]['variables'][variable] = []
        #         files_channel[new_key]['variables'][variable].append( pd.Series(files_channel[key]['variables'][variable]) )
        
        # for variable in files_channel[new_key]['variables']:
        #     files_channel[new_key]['variables'][variable] = pd.concat(files_channel[new_key]['variables'][variable])


In [14]:
# weight functions
spline_file = '/data/ana/Diffuse/NNMFit/MCEq_splines/v1.2.1/MCEq_splines_PRI-Gaisser-H4a_INT-SIBYLL23c_allfluxes.pickle'

# conventional            
flux_keys_conv =  ['conv_antinumu','conv_numu','conv_antinue','conv_nue','conv_antinutau','conv_nutau']
spline_object_conv = SplineHandler(spline_file, flux_keys_conv)
conv_flux = spline_object_conv.return_weight
generator_conv = lambda pdgid, energy, cos_zen: conv_flux(pdgid, energy, cos_zen)

# prompt
flux_keys_pr =  ['pr_antinumu','pr_numu','pr_antinue','pr_nue','pr_antinutau','pr_nutau']
spline_object_pr = SplineHandler(spline_file, flux_keys_pr)
pr_flux = spline_object_pr.return_weight
generator_pr = lambda pdgid, energy, cos_zen: pr_flux(pdgid, energy, cos_zen)

# astro
gamma_astro = 2.87
per_flavor_norm = 2.12
def AstroFluxModel(pdgid, energy, cos_zen):
    flux = 0.5*(per_flavor_norm*1e-18)*(energy/1e5)**-gamma_astro
    return flux



check if merging works now, seems to be fine

In [22]:
# did the merging go well?
data = {}

for channel, files_channel in files.items():
    channel_data = {}

    for flavor in ["NuTau", "NuTau_E2_1", "NuTau_E3_1","NuTau_E2_2", "NuTau_E3_2"]:
        weights = files_channel[flavor]["weighter"].get_weights(AstroFluxModel) * livetime_s
        rate = np.sum(weights)
        error = np.sqrt(np.sum(weights**2))
        channel_data[f"astro_{flavor}"] = f"{rate:.2f} ± {error:.2f}"
    
    data[channel] = channel_data


# Create DataFrame
df = pd.DataFrame.from_dict(data, orient='index')
print(df.to_string())

                             astro_NuTau astro_NuTau_E2_1 astro_NuTau_E3_1 astro_NuTau_E2_2 astro_NuTau_E3_2
neha_track_AllHESE           2.77 ± 0.11      2.34 ± 0.19      0.23 ± 0.02      2.62 ± 0.14      0.23 ± 0.01
neha_cascade_AllHESE        29.18 ± 0.40     28.27 ± 0.72      0.72 ± 0.03     28.54 ± 0.49      0.71 ± 0.02
neha_doublecascade_AllHESE   2.95 ± 0.09      2.49 ± 0.16      0.41 ± 0.02      2.56 ± 0.11      0.42 ± 0.01


In [17]:
data = {}

for channel, files_channel in files.items():
    channel_data = {}

    for flavor in ['NuE', "NuMu", "NuTau"]:
        weights = files_channel[flavor]["weighter"].get_weights(AstroFluxModel) * livetime_s
        rate = np.sum(weights)
        error = np.sqrt(np.sum(weights**2))
        channel_data[f"astro_{flavor}"] = f"{rate:.2f} ± {error:.2f}"

    # Conventional
    weights_conv = files_channel[flavor]["weighter"].get_weights(generator_conv) * livetime_s
    rate_conv = np.sum(weights_conv)
    err_conv = np.sqrt(np.sum(weights_conv**2))
    channel_data["conv"] = f"{rate_conv:.2f} ± {err_conv:.2f}"

    # Prompt
    weights_prompt = files_channel[flavor]["weighter"].get_weights(generator_pr) * livetime_s
    rate_prompt = np.sum(weights_prompt)
    err_prompt = np.sqrt(np.sum(weights_prompt**2))
    channel_data["prompt"] = f"{rate_prompt:.2f} ± {err_prompt:.2f}"

    data[channel] = channel_data

# Create DataFrame
df = pd.DataFrame.from_dict(data, orient='index')

# Optional: specify column order
columns_order = [f"astro_{flavor}" for flavor in ['NuE', 'NuMu', 'NuTau']] + ["conv", "prompt"]
df = df[columns_order]

# --- Add total row ---

def parse_value_error(s):
    """Parses 'value ± error' and returns (value, error) as floats."""
    match = re.match(r"([0-9.eE+-]+)\s*±\s*([0-9.eE+-]+)", s)
    return (float(match[1]), float(match[2])) if match else (0.0, 0.0)

channels_to_sum = ["neha_track_AllHESE", "neha_cascade_AllHESE", "neha_doublecascade_AllHESE"]
total_row = {}

for col in df.columns:
    total_val = 0.0
    total_err_sq = 0.0
    for ch in channels_to_sum:
        val, err = parse_value_error(df.loc[ch, col])
        total_val += val
        total_err_sq += err**2
    total_err = np.sqrt(total_err_sq)
    total_row[col] = f"{total_val:.2f} ± {total_err:.2f}"

df.loc['total'] = total_row

# Display as string table
print(df.to_string())

                               astro_NuE    astro_NuMu   astro_NuTau         conv       prompt
neha_track_AllHESE           1.82 ± 0.10  11.80 ± 0.17   2.77 ± 0.11  0.00 ± 0.00  0.06 ± 0.00
neha_cascade_AllHESE        53.85 ± 0.55   8.27 ± 0.15  29.18 ± 0.40  0.00 ± 0.00  0.68 ± 0.01
neha_doublecascade_AllHESE   1.10 ± 0.07   0.35 ± 0.03   2.95 ± 0.09  0.00 ± 0.00  0.06 ± 0.00
total                       56.77 ± 0.56  20.42 ± 0.23  34.90 ± 0.42  0.00 ± 0.00  0.80 ± 0.01


In [18]:
# # reconstructed energy
# bins = np.linspace(4.77815,7.1,14)
# selection_name = "RecoETot>60TeV"
# variable = "RecoETot"

# plotting_path = f"{main_plotting_path}/{variable}"
# os.system(f"mkdir -p {plotting_path}")

# for channel,files_channel in files.items():

#     fig, ax = plt.subplots(1, 1,figsize =(8,7))

#     for flavor in ['NuE', "NuMu", "NuTau"]:
#         selection = selection_mask( files_channel[flavor] )[selection_name]
#         y = np.log10(files_channel[flavor]['variables'][variable][ selection ])
#         w = files_channel[flavor]['variables']['Weights_Astro'][selection]

#         hist,hist_err = make_hist_error(samples=y, bins=bins,weights=w*livetime_s)

#         label = f"{flavor} : {sum(hist):.3f}"

#         plot_hist_errorbar(ax=ax,hist=hist,bins=bins,yerror=hist_err,label=label)
    
#     # atmospheric neutrinos
#     selection = selection_mask( files_channel["NuAll"] ) [selection_name]
#     y = np.log10(files_channel['NuAll']['variables'][variable][ selection ])
#     w = files_channel['NuAll']['variables']['Weights_Atmospheric'][selection]
#     hist,hist_err = make_hist_error(samples=y, bins=bins,weights=w*livetime_s)
#     label = f"Atmospheric : {sum(hist):.3f}"
#     plot_hist_errorbar(ax=ax,hist=hist,bins=bins,yerror=hist_err,label=label)

#     ax.legend()
#     ax.set_ylabel(f"Events / {livetime_yr:.2f} yr",fontdict=font_axis_label)
#     ax.set_xlabel("Reconstructed deposited energy [log10(GeV)]",fontdict=font_axis_label)
#     ax.set_title(f"Selected as {channel}")
#     # plt.savefig(f"{plotting_path}/{variable}_selection-NehaHESE.pdf")
